# Import Required Libraries

In [1]:
from sklearn.model_selection import StratifiedKFold
from model_selection import ClusteredStratifiedKFold

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

from scipy.stats import ks_2samp

from model import OptimizedKMeans
from model import GeneticProfiling
from model import GeneticClustering
from model import dae_wrapper
from model import Dense
from model import ConvDense

from constants import FISH_VARIABLE_NAMES

from correlation import select_genes, select_genes_mic
from util import to_data_frame
from itertools import compress
from datetime import datetime

import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle
import os

# Loading Data

In [2]:
clinical = pd.read_csv('data/clinical.tsv', sep='\t', index_col='ID')
clinical.iloc[:4,:2]

,response_best_response_first_line,response_days_to_disease_progression
ID,,
MMRF1007,0.0,0.0
MMRF1011,0.0,1.0
MMRF1013,1.0,1.0
MMRF1014,0.0,NaN


In [3]:
clinical = pd.read_csv('data/clinical.tsv', sep='\t', index_col='ID')

RESP_VAR_NAME = 'response_best_response_first_line'

clinical.dropna(subset=[RESP_VAR_NAME], inplace=True)

# del clinical['response_best_response_first_line']
del clinical['response_days_to_disease_progression']
del clinical['response_days_to_first_response']
del clinical['response_best_response_and_days_to_first_therapy']

#for var in FISH_VARIABLE_NAMES:
#    del clinical[var]

genefpkm = pd.read_csv('data/gene_fpkm.tsv', sep='\t', index_col='ID')

selected_index = clinical.join(genefpkm, how='inner').index

clinical = clinical.loc[selected_index,:]

clinical[RESP_VAR_NAME] = clinical[RESP_VAR_NAME].astype(int)

therapy_class = clinical['therapy_first_line_class']

del clinical['therapy_first_line_class']

# remove patients with more than five missing clinical variables
# clinical = clinical[clinical.isna().T.sum() < 10]

genefpkm = genefpkm.loc[selected_index,:]

for g in genefpkm.loc[:, genefpkm.sum() == 0].columns:
    del genefpkm[g]

genefpkm = genefpkm.dropna(axis=1, how='any')

print("Gene expressions {}".format(genefpkm.shape))
    
clinical.iloc[:6,:6]

Gene expressions (724, 49387)


,response_best_response_first_line,cmmc,ecog_ps,cell_markers,percent_aneuploid,percent_plama_cells_bone_marrow
ID,,,,,,
MMRF1021,0,NaN,PS 1 (Restricted in physically strenuous activ...,CD13,0.0,4.9
MMRF1024,0,NaN,PS 1 (Restricted in physically strenuous activ...,CD117,11.0,6.0
MMRF1029,0,NaN,PS 1 (Restricted in physically strenuous activ...,CD117,0.0,8.4
MMRF1030,1,NaN,PS 1 (Restricted in physically strenuous activ...,CD117,15.4,9.6
MMRF1031,0,NaN,PS 0 (Fully Active),CD117,18.3,10.1
MMRF1032,0,NaN,PS 2 (Ambulatory and capable of all selfcare),CD117,20.7,11.1


In [4]:
for c in clinical.columns:
    print('{} ({}): {}'.format(c, clinical[c].dtype, list(clinical[c].unique())[:4]))

response_best_response_first_line (int32): [0, 1]
cmmc (float64): [nan, 5913.0, 22169.0, 3864.0]
ecog_ps (object): ['PS 1 (Restricted in physically strenuous activity)', 'PS 0 (Fully Active)', 'PS 2 (Ambulatory and capable of all selfcare)', nan]
cell_markers (object): ['CD13', 'CD117', 'CD138', nan]
percent_aneuploid (float64): [0.0, 11.0, 15.4, 18.3]
percent_plama_cells_bone_marrow (float64): [4.9, 6.0, 8.4, 9.6]
percent_plama_cells_peripherical_blood (float64): [0.0, 0.1, 0.6, 0.03]
creatinine (float64): [88.4, 123.76, 106.08, 55.692]
iss (float64): [1.0, 2.0, 3.0, nan]
absolute_neutrophil (float64): [2.4, 2.3, 2.6, 2.5]
platelet (float64): [216.0, 188.0, 219.0, 215.0]
wbc_x10_10_9_l (float64): [5.2, 4.3, 4.0, 4.7]
bun (float64): [8.925, 11.424, 5.355, nan]
glucose (float64): [4.675, 4.785, 5.995, 6.27]
total_protein (float64): [11.5, 8.7, 9.4, 9.8]
albumin (float64): [39.0, 40.0, 36.0, 37.0]
beta_2_microglobulin (float64): [2.1, 3.61, 1.9, 1.98]
calcium (float64): [2.4, 2.45, 2.25,

In [5]:
from collections import Counter
print(dict(Counter(clinical[RESP_VAR_NAME])))
clinical.head()

{0: 553, 1: 171}


,response_best_response_first_line,cmmc,ecog_ps,cell_markers,percent_aneuploid,percent_plama_cells_bone_marrow,percent_plama_cells_peripherical_blood,creatinine,iss,absolute_neutrophil,...,t_8_14_mafa,t_8_14_myc,lga,lgg,lgl_kappa,lgl_lambda,lgm,m_protein,therapy_first_line,first_line_transplant
ID,,,,,,,,,,,,,,,,,,,,,
MMRF1021,0,NaN,PS 1 (Restricted in physically strenuous activ...,CD13,0.0,4.9,0.0,88.400,1.0,2.40,...,Not Detected,Not Detected,0.66,70.50,48.57,3.280,0.40,3.05,Bor-Len-Dex,Yes
MMRF1024,0,NaN,PS 1 (Restricted in physically strenuous activ...,CD117,11.0,6.0,0.0,123.760,2.0,2.30,...,NaN,NaN,NaN,NaN,9.66,0.870,NaN,2.60,NaN,No
MMRF1029,0,NaN,PS 1 (Restricted in physically strenuous activ...,CD117,0.0,8.4,0.0,106.080,1.0,2.60,...,Not Detected,Not Detected,0.69,27.99,27.04,0.740,0.43,1.80,Bor-Len-Dex,No
MMRF1030,1,NaN,PS 1 (Restricted in physically strenuous activ...,CD117,15.4,9.6,0.0,55.692,1.0,2.50,...,Not Detected,Detected,0.24,41.63,NaN,7.300,0.23,3.55,Bor-Len-Dex,Yes
MMRF1031,0,NaN,PS 0 (Fully Active),CD117,18.3,10.1,0.0,81.328,1.0,10.29,...,Not Detected,Not Detected,15.20,6.47,23.59,1.166,0.76,1.52,Bor-Len-Dex,No


# Transforming Qualitative Variables into Dummy Ones

In [6]:
clinical['first_line_transplant'] = clinical['first_line_transplant'].replace('Yes', 1).replace('No', 0)

for column in clinical:
    
    values = clinical[column]
    
    if values.dtype == 'object':
        
        values = pd.get_dummies(values)
        
        n_values = values.shape[1]
        
        values.columns = [column + '_' + str(c).lower().replace(' ', '_') for c in values.columns]
    
        del clinical[column]
        
        if n_values == 2:
            values = values.iloc[:, [0]]
        
        clinical = clinical.join(values, how='inner')

clinical = clinical.fillna(0)

print('Clinical data set with {} samples and {} features'.format(*clinical.shape))
clinical.iloc[:8,:]

Clinical data set with 724 samples and 71 features


,response_best_response_first_line,cmmc,percent_aneuploid,percent_plama_cells_bone_marrow,percent_plama_cells_peripherical_blood,creatinine,iss,absolute_neutrophil,platelet,wbc_x10_10_9_l,...,t_4_14_whsc1_detected,t_6_14_ccnd3_detected,t_8_14_mafa_detected,t_8_14_myc_detected,therapy_first_line_bor,therapy_first_line_bor-cyc-dex,therapy_first_line_bor-dex,therapy_first_line_bor-len-dex,therapy_first_line_len,therapy_first_line_len-dex
ID,,,,,,,,,,,,,,,,,,,,,
MMRF1021,0,0.0,0.0,4.9,0.0,88.400,1.0,2.40,216.0,5.2,...,1,0,0,0,0,0,0,1,0,0
MMRF1024,0,0.0,11.0,6.0,0.0,123.760,2.0,2.30,188.0,4.3,...,0,0,0,0,0,0,0,0,0,0
MMRF1029,0,0.0,0.0,8.4,0.0,106.080,1.0,2.60,219.0,4.0,...,0,0,0,0,0,0,0,1,0,0
MMRF1030,1,0.0,15.4,9.6,0.0,55.692,1.0,2.50,215.0,4.7,...,0,0,0,1,0,0,0,1,0,0
MMRF1031,0,0.0,18.3,10.1,0.0,81.328,1.0,10.29,385.0,12.4,...,0,0,0,0,0,0,0,1,0,0
MMRF1032,0,0.0,20.7,11.1,0.0,70.720,2.0,1.30,166.0,2.5,...,0,0,0,0,0,1,0,0,0,0
MMRF1033,0,0.0,18.5,12.0,0.0,79.560,1.0,3.99,307.0,7.4,...,0,0,0,0,0,0,0,0,0,1
MMRF1037,0,0.0,20.7,17.0,0.0,70.720,1.0,3.20,361.0,5.4,...,0,0,0,0,0,0,0,0,0,1


In [7]:
genefpkm.iloc[:8,:8]

,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,ENSG00000000971,ENSG00000001036
ID,,,,,,,,
MMRF1021,17.984200,0.0,36.6665,6.29140,4.54056,0.401168,43.298500,25.6721
MMRF1024,11.962400,0.0,22.0121,5.38688,1.06560,4.072600,0.092420,21.3765
MMRF1029,17.651800,0.0,38.2082,6.61449,2.55321,0.237359,0.158369,55.2849
MMRF1030,1.595960,0.0,35.0286,4.62864,1.31648,0.642315,0.177913,40.7695
MMRF1031,0.030286,0.0,42.3708,5.34199,5.05216,0.439662,1.604640,63.2392
MMRF1032,0.681066,0.0,30.2787,3.01076,1.62869,0.587525,1.091920,20.0476
MMRF1033,0.595213,0.0,29.5319,6.10217,1.29061,2.331920,0.364109,25.0889
MMRF1037,0.513027,0.0,29.3762,4.32193,1.62923,0.131227,0.401753,26.2385


# Removing Bias from Therapy

In [8]:
therapy_columns = []

for c in clinical.columns:
    if 'therapy_first_line' in c:
        therapy_columns.append(c)

to_delete = []

for a in list(clinical[therapy_columns].loc[:,clinical[therapy_columns].sum() < 10].columns):
    to_delete += list(clinical.loc[clinical[a] == 1,:].index)
    del clinical[a]
    
clinical = clinical.loc[~clinical.index.isin(to_delete),:]

genefpkm = genefpkm.loc[~genefpkm.index.isin(to_delete),:]

therapy_columns = [t for t in therapy_columns if t in clinical.columns]

print('Valid Therapies')
for t in therapy_columns:
    print('* {}'.format(t.replace('therapy_first_line_', '')))

Valid Therapies
* bor-cyc-dex
* bor-dex
* bor-len-dex
* len-dex


In [9]:
%matplotlib inline

from collections import Counter
if False:
    for c in selected_feats:
        print(clinical[c].unique())
        print(Counter(clinical[c]))

        def fff(x):
            vvv = clinical[c].quantile([0, .1,.2,.3,.4,.5,.6,.7,.8,.9,1.]).values
            for i, (a, b) in enumerate(zip(vvv[:-1], vvv[1:])):
                if a <= x < b:
                    return i
            return 10

        clinical['{}_cat'.format(c)] = clinical[c].apply(fff)

        clinical['{}_cat'.format(c)].hist()

# Training Process

In [12]:
from collections import Counter
from scipy.special import erfinv
from sklearn.preprocessing import StandardScaler
from optimization import lightgbm_optimizer
from evaluation import optimize_threshold, classification_metrics, ks_score

import time

RANDOM_STATE = 10

x, y = clinical.values[:, 1:], clinical.values[:, 0]

kfold = StratifiedKFold(5, shuffle=True, random_state=RANDOM_STATE)

inference_columns = ['ID', 'y_hat', 'y_true', 'n_feats', 
                     'fold', 'therapy', 'threshold']
inference_train = {c: [] for c in inference_columns}
inference_valid = {c: [] for c in inference_columns}

result = {c: [] for c in [
    'fold', 'n_feats', 'n_genes', 'auc_train', 'auc_valid', 'tp', 'tn', 'fp', 'fn', 'optimal_threshold',
    
    'accuracy', 'sensitivity', 'specificity', 'precision', 'ks',
    
    'optimization_n_folds', 'optimization_n_calls',
    
    'overall_time', 'gene_normalization_time', 'clinical_normalization_time', 'feature_selection_time', 
    'genetic_profiling_time', 'gene_clustering_time', 'dda_time', 'baysian_optimization_time', 'lightgbm_train_time',
    
    'train_test_distance_avg', 'train_test_distance_std', 'train_test_distance_min', 'train_test_distance_max',
    
    'learning_rate', 'num_leaves', 'max_depth', 'scale_pos_weight', 'min_child_weight', 'colsample_bytree',
    'min_split_gain', 'min_child_samples', 'subsample', 'bin_construct_sample_cnt',
    'y_train_hat_min', 'y_train_hat_max']}

def therapy_from_dummy(row):
    global therapy_columns
    try:
        return therapy_columns[row.tolist().index(1)]
    except Exception as e:
        return None
    

for fold, (train_index, valid_index) in enumerate(kfold.split(x, y)):
    
    fold += 1

    print('Fold {}'.format(fold))
    
    #######################################################################################################
    # Split train & valid
    #######################################################################################################
    
    start = time.time()

    response_train = clinical.iloc[train_index, [0]]
    response_valid = clinical.iloc[valid_index, [0]]

    clinical_train_ = clinical.iloc[train_index, 1:]
    clinical_valid_ = clinical.iloc[valid_index, 1:]

    genefpkm_scaler_minmax = MinMaxScaler()

    genefpkm_train_ = genefpkm.iloc[train_index, :]
    genefpkm_train_ = pd.DataFrame(genefpkm_scaler_minmax.fit_transform(genefpkm_train_.values), 
        columns=genefpkm_train_.columns, index=genefpkm_train_.index)

    genefpkm_valid_ = genefpkm.iloc[valid_index, :]
    genefpkm_valid_ = pd.DataFrame(genefpkm_scaler_minmax.transform(
        genefpkm_valid_.values), columns=genefpkm_valid_.columns, index=genefpkm_valid_.index)

    gene_normalization_time = time.time() - start

    #######################################################################################################
    # Select gene expressions
    #######################################################################################################
    
    start = time.time()

    selected_genes = select_genes(genefpkm_train_, response_train.iloc[:,0].values, threshold=.050)
    selected_feats = select_genes(clinical_train_, response_train.iloc[:,0].values, threshold=.001)
    
    # export feature selection result
    pd.DataFrame({'gene': selected_genes[0], 
                  'pvalue': selected_genes[1]}).to_csv(
        'output/brfl/selected_genes_fold_{}.csv'.format(fold))
    pd.DataFrame({'gene': selected_feats[0], 
                  'pvalue': selected_feats[1]}).to_csv(
        'output/brfl/selected_feats_fold_{}.csv'.format(fold))
        
    selected_genes = selected_genes[0]
    selected_feats = selected_feats[0]
    
    # force therapy columns to be selected
    selected_feats = list(set(selected_feats + therapy_columns))

    feature_selection_time = time.time() - start
    
    for n_features in range(20, 41):
   
        overall_start = time.time()

        #######################################################################################################
        # Remove unselected features
        #######################################################################################################
    
        clinical_train = clinical_train_.loc[:,selected_feats].copy()
        clinical_valid = clinical_valid_.loc[:,selected_feats].copy()
    
        genefpkm_train = genefpkm_train_.loc[:,selected_genes[:n_features]].copy()
        genefpkm_valid = genefpkm_valid_.loc[:,selected_genes[:n_features]].copy()

        #######################################################################################################
        # Train & Test distances
        #######################################################################################################
        
        dists = []

        for row_train in clinical_train.join(genefpkm_train, how='inner').values:
            for row_valid in clinical_valid.join(genefpkm_valid, how='inner').values:
                dists.append(np.linalg.norm(row_train-row_valid))
        
        train_test_distance_avg = np.mean(dists)
        train_test_distance_std = np.std(dists)
        train_test_distance_min = np.min(dists)
        train_test_distance_max = np.max(dists)

        #######################################################################################################
        # Genetic Profiling
        #######################################################################################################
        
        start = time.time()

        genetic_profiling = GeneticProfiling(random_state=RANDOM_STATE)

        genetic_profiling.fit(genefpkm_train)

        with open('output/brfl/kmeans_genetic_profiling_fold_{}.pkl'.format(fold), 'wb') as file:
            pickle.dump(genetic_profiling, file)

        profiling_train = to_data_frame(genetic_profiling.transform(genefpkm_train), 
                                        prefix='PV', index=genefpkm_train.index)    
        clinical_train = pd.concat([clinical_train, profiling_train], axis=1)

        profiling_valid = to_data_frame(genetic_profiling.transform(genefpkm_valid), 
                                        prefix='PV', index=genefpkm_valid.index)
        clinical_valid = pd.concat([clinical_valid, profiling_valid], axis=1)    

        genetic_profiling_time = time.time() - start

        #######################################################################################################
        # Gene Clustering
        #######################################################################################################
        
        start = time.time()

        genetic_clustering = GeneticClustering(random_state=RANDOM_STATE, verbose=0, early_stopping_rounds=10)

        genetic_clustering.fit(genefpkm_train)

        with open('output/brfl/kmeans_genetic_clustering_fold_{}.pkl'.format(fold), 'wb') as file:
            pickle.dump(genetic_clustering, file)
            
        gene_cluster_train = to_data_frame(genetic_clustering.transform(genefpkm_train), 
                                           prefix='GC', index=genefpkm_train.index)
        gene_cluster_valid = to_data_frame(genetic_clustering.transform(genefpkm_valid), 
                                           prefix='GC', index=genefpkm_valid.index)        
        
        clinical_train = pd.concat([clinical_train, gene_cluster_train], axis=1)
        clinical_valid = pd.concat([clinical_valid, gene_cluster_valid], axis=1)

        gene_clustering_time = time.time() - start

        #######################################################################################################
        # Normalizing Clinical Data
        #######################################################################################################

        start = time.time()
        
        clinical_scaler_minmax = MinMaxScaler()

        clinical_train__ = clinical_scaler_minmax.fit_transform(clinical_train)
        clinical_train = pd.DataFrame(clinical_train__, index=clinical_train.index, columns=clinical_train.columns)
        clinical_train = clinical_train.fillna(0)

        clinical_valid__ = clinical_scaler_minmax.transform(clinical_valid)
        clinical_valid = pd.DataFrame(clinical_valid__, index=clinical_valid.index, columns=clinical_valid.columns)
        clinical_valid = clinical_valid.fillna(0)

        clinical_normalization_time = time.time() - start
        
        #######################################################################################################
        # Denoising Autoencoder
        #######################################################################################################

        start = time.time()
        
        dda_train, dda_valid = dae_wrapper(genefpkm_train, genefpkm_valid, RANDOM_STATE, '{}_{}_genexp'.format(fold, n_features))

        dda_scaler_minmax = MinMaxScaler()

        dda_train = dda_scaler_minmax.fit_transform(dda_train)
        dda_train = pd.DataFrame(dda_train, index=genefpkm_train.index)
        dda_train.columns = [str(col) + '_DDA' for col in genefpkm_train.columns]

        dda_valid = dda_scaler_minmax.transform(dda_valid)
        dda_valid = pd.DataFrame(dda_valid, index=genefpkm_valid.index)
        dda_valid.columns = [str(col) + '_DDA' for col in genefpkm_valid.columns]

        dda_time = time.time() - start
        
        #######################################################################################################
        # Joining all features
        #######################################################################################################
        
        x_train = clinical_train.join(genefpkm_train, how='inner').join(dda_train, how='inner')
        x_valid = clinical_valid.join(genefpkm_valid, how='inner').join(dda_valid, how='inner')
        
        # x_train = x_train.loc[x_train[therapy_columns].astype(int)
        # .apply(lambda x: np.sum(x.tolist()), axis=1) != 0]
        # response_train = response_train.loc[(x_train[therapy_columns]
        # .astype(int).apply(lambda x: np.sum(x.tolist()), axis=1) != 0).values]
        
        # x_valid = x_valid.loc[x_valid[therapy_columns].astype(int)
        # .apply(lambda x: np.sum(x.tolist()), axis=1) != 0]
        # response_valid = response_valid.loc[(x_valid[therapy_columns]
        # .astype(int).apply(lambda x: np.sum(x.tolist()), axis=1) != 0).values]
        
        #######################################################################################################
        # Baysian Optimization
        #######################################################################################################
        
        start = time.time()

        file_name = 'output/brfl/optimization_lgbm_fold_{}.pkl'.format(fold)
        
        optimization_n_folds, optimization_n_calls = 2, 10
        
        opt = lightgbm_optimizer(x_train.values, response_train.values, 
                                 nfolds=optimization_n_folds, n_calls=optimization_n_calls, 
                                 random_state=RANDOM_STATE).x;

        params = {
            'learning_rate': opt[0],
            'num_leaves': opt[1],
            'max_depth': opt[2],
            'scale_pos_weight': opt[3],
            'min_child_weight': opt[4],
            'colsample_bytree': opt[5],
            'min_split_gain': opt[6],
            'min_child_samples': opt[7],
            'subsample': opt[8],
            'bin_construct_sample_cnt': opt[9],

            'objective':'binary',
            'metric':'auc',
            'is_unbalance':False,
            'nthread':24,          
            'verbose': -1,
            'device': 'gpu',
            'gpu_platform_id': 1,
            'gpu_device_id': 0,
            'random_state': RANDOM_STATE}

        baysian_optimization_time = time.time() - start

        #######################################################################################################
        # Light GBM Train
        #######################################################################################################
        
        start = time.time()

        lgb_train = lgb.Dataset(x_train, response_train)
        lgb_valid = lgb.Dataset(x_valid, response_valid)

        gbm = lgb.train(params, lgb_train, valid_sets=lgb_valid, num_boost_round=100000, 
                        early_stopping_rounds=200, verbose_eval=False)
        
        lightgbm_train_time = time.time() - start
        
        #######################################################################################################
        # Light GBM Inference
        #######################################################################################################

        y_train_hat = gbm.predict(x_train.values, num_iteration=gbm.best_iteration)
        y_valid_hat = gbm.predict(x_valid.values, num_iteration=gbm.best_iteration)
        
        #######################################################################################################
        # Performance Evaluation
        #######################################################################################################
        
        optimal_threshold = optimize_threshold(response_train, y_train_hat)
        if optimal_threshold is None:
            optimal_threshold = np.median(y_train_hat)
        y_valid_optimized = [int(y_ > optimal_threshold) for y_ in y_valid_hat]
        
        tn, fp, fn, tp = confusion_matrix(response_valid, y_valid_optimized).ravel()
        
        metrics = classification_metrics(tn, fp, fn, tp)
        ks = ks_score(response_valid, y_valid_hat)
        
        auc_train = roc_auc_score(response_train, y_train_hat)
        auc_valid = roc_auc_score(response_valid, y_valid_hat)
        
        overall_time = time.time() - overall_start
        
        ########################################################################################################
        # Save inference 
        ########################################################################################################
        
        # ['ID', 'y_hat', 'y_true', 'fold', 'n_feats', 'therapy', 'threshold']
        
        inference_train['ID'] += list(x_train.index)
        inference_train['y_true'] += list(response_train.values.reshape((-1,)))
        inference_train['y_hat'] += list(y_train_hat)
        inference_train['therapy'] += x_train[therapy_columns].astype(int).apply(therapy_from_dummy, axis=1).tolist()
        inference_train['threshold'] += [optimal_threshold] * len(list(x_train.index))
        inference_train['fold'] += [fold] * len(list(x_train.index))
        inference_train['n_feats'] += [n_features] * len(list(x_train.index))
        
        inference_valid['ID'] += list(x_valid.index)
        inference_valid['y_true'] += list(response_valid.values.reshape((-1,)))
        inference_valid['y_hat'] += list(y_valid_hat)
        inference_valid['therapy'] += x_valid[therapy_columns].astype(int).apply(therapy_from_dummy, axis=1).tolist()
        inference_valid['threshold'] += [optimal_threshold] * len(list(x_valid.index))
        inference_valid['fold'] += [fold] * len(list(x_valid.index))
        inference_valid['n_feats'] += [n_features] * len(list(x_valid.index))
        
        ########################################################################################################
        # Save fold metadata        
        ########################################################################################################
        result['fold'].append(fold)
        result['n_feats'].append(len(selected_feats))
        result['n_genes'].append(n_features)
        result['auc_train'].append(auc_train)
        result['auc_valid'].append(auc_valid)
        result['optimal_threshold'].append(optimal_threshold)
        result['tn'].append(tn)
        result['fp'].append(fp)
        result['fn'].append(fn)
        result['tp'].append(tp)
        result['ks'].append(ks)
        result['accuracy'].append(metrics['accuracy'])
        result['sensitivity'].append(metrics['sensitivity'])
        result['precision'].append(metrics['precision'])
        result['specificity'].append(metrics['specificity'])
        result['overall_time'].append(overall_time)
        result['lightgbm_train_time'].append(lightgbm_train_time)
        result['baysian_optimization_time'].append(baysian_optimization_time)
        result['dda_time'].append(dda_time)
        result['clinical_normalization_time'].append(clinical_normalization_time)
        result['gene_clustering_time'].append(gene_clustering_time)
        result['genetic_profiling_time'].append(genetic_profiling_time)
        result['feature_selection_time'].append(feature_selection_time)
        result['gene_normalization_time'].append(gene_normalization_time)
        result['optimization_n_folds'].append(optimization_n_folds)
        result['optimization_n_calls'].append(optimization_n_calls)
        result['learning_rate'].append(opt[0])
        result['num_leaves'].append(opt[1])
        result['max_depth'].append(opt[2])
        result['scale_pos_weight'].append(opt[3])
        result['min_child_weight'].append(opt[4])
        result['colsample_bytree'].append(opt[5])
        result['min_split_gain'].append(opt[6])
        result['min_child_samples'].append(opt[7])
        result['subsample'].append(opt[8])
        result['bin_construct_sample_cnt'].append(opt[9])
        result['train_test_distance_avg'].append(train_test_distance_avg)
        result['train_test_distance_std'].append(train_test_distance_std)
        result['train_test_distance_min'].append(train_test_distance_min)
        result['train_test_distance_max'].append(train_test_distance_max)
        result['y_train_hat_min'].append(min(y_train_hat))
        result['y_train_hat_max'].append(max(y_train_hat))
        
        print(fold, n_features, auc_train, auc_valid)

pd.DataFrame(inference_train).to_csv('output/inference_train_5_fold.csv')
pd.DataFrame(inference_valid).to_csv('output/inference_valid_5_fold.csv')
pd.DataFrame(result).to_csv('output/result_5_fold.csv')

Fold 1
1 20 0.9959601081812035 0.6343381389252948
1 21 0.9999492900608519 0.6062909567496724
1 22 0.9969405003380662 0.6222804718217562
1 23 1.0 0.6251638269986893
1 24 1.0 0.6319790301441678
1 25 1.0 0.6482306684141547
1 26 1.0 0.6023591087811271
1 27 0.9941514536849223 0.682175622542595
1 28 0.9996112237998648 0.6146788990825688
1 29 1.0 0.598951507208388
1 30 1.0 0.6448230668414154
1 31 0.9999830966869506 0.63564875491481
1 32 0.9999323867478026 0.6579292267365662
1 33 0.7369421906693713 0.7003931847968545
1 34 0.9999661933739014 0.6235910878112713
1 35 1.0 0.6857142857142857
1 36 1.0 0.6264744429882044
1 37 1.0 0.6123197903014417
1 38 0.9339925625422583 0.611926605504587
1 39 1.0 0.5981651376146789
1 40 1.0 0.6398427260812581
Fold 2
2 20 0.79066196828593 0.6486778197517539
2 21 0.9985065861229969 0.6025364274150027
2 22 0.7655424112761138 0.6218294657312466
2 23 0.9975333501132646 0.6189962223421479
2 24 1.0 0.6262817053426875
2 25 1.0 0.6570426335671884
2 26 0.79970635120396 0.651

In [11]:
from collections import Counter
from scipy.special import erfinv
from sklearn.preprocessing import StandardScaler
from optimization import lightgbm_optimizer
from evaluation import optimize_threshold, classification_metrics, ks_score

import time

RANDOM_STATE = 10

x, y = clinical.values[:, 1:], clinical.values[:, 0]

kfold = StratifiedKFold(10, shuffle=True, random_state=RANDOM_STATE)

inference_columns = ['ID', 'y_hat', 'y_true', 'n_feats', 'fold', 'therapy', 'threshold']
inference_train = {c: [] for c in inference_columns}
inference_valid = {c: [] for c in inference_columns}

result = {c: [] for c in [
    'fold', 'n_feats', 'n_genes', 'auc_train', 'auc_valid', 'tp', 'tn', 'fp', 'fn', 'optimal_threshold',
    
    'accuracy', 'sensitivity', 'specificity', 'precision', 'ks',
    
    'optimization_n_folds', 'optimization_n_calls',
    
    'overall_time', 'gene_normalization_time', 'clinical_normalization_time', 'feature_selection_time', 
    'genetic_profiling_time', 'gene_clustering_time', 'dda_time', 'baysian_optimization_time', 'lightgbm_train_time',
    
    'train_test_distance_avg', 'train_test_distance_std', 'train_test_distance_min', 'train_test_distance_max',
    
    'learning_rate', 'num_leaves', 'max_depth', 'scale_pos_weight', 'min_child_weight', 'colsample_bytree',
    'min_split_gain', 'min_child_samples', 'subsample', 'bin_construct_sample_cnt',
    'y_train_hat_min', 'y_train_hat_max']}

def therapy_from_dummy(row):
    global therapy_columns
    try:
        return therapy_columns[row.tolist().index(1)]
    except Exception as e:
        return None
    

for fold, (train_index, valid_index) in enumerate(kfold.split(x, y)):
    
    fold += 1

    print('Fold {}'.format(fold))
    
    #######################################################################################################
    # Split train & valid
    #######################################################################################################
    
    start = time.time()

    response_train = clinical.iloc[train_index, [0]]
    response_valid = clinical.iloc[valid_index, [0]]

    clinical_train_ = clinical.iloc[train_index, 1:]
    clinical_valid_ = clinical.iloc[valid_index, 1:]

    genefpkm_scaler_minmax = MinMaxScaler()

    genefpkm_train_ = genefpkm.iloc[train_index, :]
    genefpkm_train_ = pd.DataFrame(genefpkm_scaler_minmax.fit_transform(genefpkm_train_.values), 
        columns=genefpkm_train_.columns, index=genefpkm_train_.index)

    genefpkm_valid_ = genefpkm.iloc[valid_index, :]
    genefpkm_valid_ = pd.DataFrame(genefpkm_scaler_minmax.transform(
        genefpkm_valid_.values), columns=genefpkm_valid_.columns, index=genefpkm_valid_.index)

    gene_normalization_time = time.time() - start

    #######################################################################################################
    # Select gene expressions
    #######################################################################################################
    
    start = time.time()

    selected_genes = select_genes(genefpkm_train_, response_train.iloc[:,0].values, threshold=.050)
    selected_feats = select_genes(clinical_train_, response_train.iloc[:,0].values, threshold=.001)
    
    # export feature selection result
    pd.DataFrame({'gene': selected_genes[0], 
                  'pvalue': selected_genes[1]}).to_csv(
        'output/brfl/selected_genes_fold_{}.csv'.format(fold))
    pd.DataFrame({'gene': selected_feats[0], 
                  'pvalue': selected_feats[1]}).to_csv(
        'output/brfl/selected_feats_fold_{}.csv'.format(fold))
        
    selected_genes = selected_genes[0]
    selected_feats = selected_feats[0]
    
    # force therapy columns to be selected
    selected_feats = list(set(selected_feats + therapy_columns))

    feature_selection_time = time.time() - start
    
    for n_features in range(20, 41):
   
        overall_start = time.time()

        #######################################################################################################
        # Remove unselected features
        #######################################################################################################
    
        clinical_train = clinical_train_.loc[:,selected_feats].copy()
        clinical_valid = clinical_valid_.loc[:,selected_feats].copy()
    
        genefpkm_train = genefpkm_train_.loc[:,selected_genes[:n_features]].copy()
        genefpkm_valid = genefpkm_valid_.loc[:,selected_genes[:n_features]].copy()

        #######################################################################################################
        # Train & Test distances
        #######################################################################################################
        
        dists = []

        for row_train in clinical_train.join(genefpkm_train, how='inner').values:
            for row_valid in clinical_valid.join(genefpkm_valid, how='inner').values:
                dists.append(np.linalg.norm(row_train-row_valid))
        
        train_test_distance_avg = np.mean(dists)
        train_test_distance_std = np.std(dists)
        train_test_distance_min = np.min(dists)
        train_test_distance_max = np.max(dists)

        #######################################################################################################
        # Genetic Profiling
        #######################################################################################################
        
        start = time.time()

        genetic_profiling = GeneticProfiling(random_state=RANDOM_STATE)

        genetic_profiling.fit(genefpkm_train)

        with open('output/brfl/kmeans_genetic_profiling_fold_{}.pkl'.format(fold), 'wb') as file:
            pickle.dump(genetic_profiling, file)

        profiling_train = to_data_frame(genetic_profiling.transform(genefpkm_train), 
                                        prefix='PV', index=genefpkm_train.index)    
        clinical_train = pd.concat([clinical_train, profiling_train], axis=1)

        profiling_valid = to_data_frame(genetic_profiling.transform(genefpkm_valid), 
                                        prefix='PV', index=genefpkm_valid.index)
        clinical_valid = pd.concat([clinical_valid, profiling_valid], axis=1)    

        genetic_profiling_time = time.time() - start

        #######################################################################################################
        # Gene Clustering
        #######################################################################################################
        
        start = time.time()

        genetic_clustering = GeneticClustering(random_state=RANDOM_STATE, verbose=0, early_stopping_rounds=10)

        genetic_clustering.fit(genefpkm_train)

        with open('output/brfl/kmeans_genetic_clustering_fold_{}.pkl'.format(fold), 'wb') as file:
            pickle.dump(genetic_clustering, file)
            
        gene_cluster_train = to_data_frame(genetic_clustering.transform(genefpkm_train), 
                                           prefix='GC', index=genefpkm_train.index)
        gene_cluster_valid = to_data_frame(genetic_clustering.transform(genefpkm_valid), 
                                           prefix='GC', index=genefpkm_valid.index)        
        
        clinical_train = pd.concat([clinical_train, gene_cluster_train], axis=1)
        clinical_valid = pd.concat([clinical_valid, gene_cluster_valid], axis=1)

        gene_clustering_time = time.time() - start

        #######################################################################################################
        # Normalizing Clinical Data
        #######################################################################################################

        start = time.time()
        
        clinical_scaler_minmax = MinMaxScaler()

        clinical_train__ = clinical_scaler_minmax.fit_transform(clinical_train)
        clinical_train = pd.DataFrame(clinical_train__, index=clinical_train.index, columns=clinical_train.columns)
        clinical_train = clinical_train.fillna(0)

        clinical_valid__ = clinical_scaler_minmax.transform(clinical_valid)
        clinical_valid = pd.DataFrame(clinical_valid__, index=clinical_valid.index, columns=clinical_valid.columns)
        clinical_valid = clinical_valid.fillna(0)

        clinical_normalization_time = time.time() - start
        
        #######################################################################################################
        # Denoising Autoencoder
        #######################################################################################################

        start = time.time()
        
        dda_train, dda_valid = dae_wrapper(genefpkm_train, genefpkm_valid, RANDOM_STATE, '{}_{}_genexp'.format(fold, n_features))

        dda_scaler_minmax = MinMaxScaler()

        dda_train = dda_scaler_minmax.fit_transform(dda_train)
        dda_train = pd.DataFrame(dda_train, index=genefpkm_train.index)
        dda_train.columns = [str(col) + '_DDA' for col in genefpkm_train.columns]

        dda_valid = dda_scaler_minmax.transform(dda_valid)
        dda_valid = pd.DataFrame(dda_valid, index=genefpkm_valid.index)
        dda_valid.columns = [str(col) + '_DDA' for col in genefpkm_valid.columns]

        dda_time = time.time() - start
        
        #######################################################################################################
        # Joining all features
        #######################################################################################################
        
        x_train = clinical_train.join(genefpkm_train, how='inner').join(dda_train, how='inner')
        x_valid = clinical_valid.join(genefpkm_valid, how='inner').join(dda_valid, how='inner')
        
        # x_train = x_train.loc[x_train[therapy_columns].astype(int)
        # .apply(lambda x: np.sum(x.tolist()), axis=1) != 0]
        # response_train = response_train.loc[(x_train[therapy_columns]
        # .astype(int).apply(lambda x: np.sum(x.tolist()), axis=1) != 0).values]
        
        # x_valid = x_valid.loc[x_valid[therapy_columns].astype(int)
        # .apply(lambda x: np.sum(x.tolist()), axis=1) != 0]
        # response_valid = response_valid.loc[(x_valid[therapy_columns]
        # .astype(int).apply(lambda x: np.sum(x.tolist()), axis=1) != 0).values]
        
        #######################################################################################################
        # Baysian Optimization
        #######################################################################################################
        
        start = time.time()

        file_name = 'output/brfl/optimization_lgbm_fold_{}.pkl'.format(fold)
        
        optimization_n_folds, optimization_n_calls = 2, 10
        
        opt = lightgbm_optimizer(x_train.values, response_train.values, 
                                 nfolds=optimization_n_folds, n_calls=optimization_n_calls, 
                                 random_state=RANDOM_STATE).x;

        params = {
            'learning_rate': opt[0],
            'num_leaves': opt[1],
            'max_depth': opt[2],
            'scale_pos_weight': opt[3],
            'min_child_weight': opt[4],
            'colsample_bytree': opt[5],
            'min_split_gain': opt[6],
            'min_child_samples': opt[7],
            'subsample': opt[8],
            'bin_construct_sample_cnt': opt[9],

            'objective':'binary',
            'metric':'auc',
            'is_unbalance':False,
            'nthread':24,          
            'verbose': -1,
            'device': 'gpu',
            'gpu_platform_id': 1,
            'gpu_device_id': 0,
            'random_state': RANDOM_STATE}

        baysian_optimization_time = time.time() - start

        #######################################################################################################
        # Light GBM Train
        #######################################################################################################
        
        start = time.time()

        lgb_train = lgb.Dataset(x_train, response_train)
        lgb_valid = lgb.Dataset(x_valid, response_valid)

        gbm = lgb.train(params, lgb_train, valid_sets=lgb_valid, num_boost_round=100000, 
                        early_stopping_rounds=200, verbose_eval=False)
        
        lightgbm_train_time = time.time() - start
        
        #######################################################################################################
        # Light GBM Inference
        #######################################################################################################

        y_train_hat = gbm.predict(x_train.values, num_iteration=gbm.best_iteration)
        y_valid_hat = gbm.predict(x_valid.values, num_iteration=gbm.best_iteration)
        
        #######################################################################################################
        # Performance Evaluation
        #######################################################################################################
        
        optimal_threshold = optimize_threshold(response_train, y_train_hat)
        if optimal_threshold is None:
            optimal_threshold = np.median(y_train_hat)
        y_valid_optimized = [int(y_ > optimal_threshold) for y_ in y_valid_hat]
        
        tn, fp, fn, tp = confusion_matrix(response_valid, y_valid_optimized).ravel()
        
        metrics = classification_metrics(tn, fp, fn, tp)
        ks = ks_score(response_valid, y_valid_hat)
        
        auc_train = roc_auc_score(response_train, y_train_hat)
        auc_valid = roc_auc_score(response_valid, y_valid_hat)
        
        overall_time = time.time() - overall_start
        
        ########################################################################################################
        # Save inference 
        ########################################################################################################
        
        # ['ID', 'y_hat', 'y_true', 'fold', 'n_feats', 'therapy', 'threshold']
        
        inference_train['ID'] += list(x_train.index)
        inference_train['y_true'] += list(response_train.values.reshape((-1,)))
        inference_train['y_hat'] += list(y_train_hat)
        inference_train['therapy'] += x_train[therapy_columns].astype(int).apply(therapy_from_dummy, axis=1).tolist()
        inference_train['threshold'] += [optimal_threshold] * len(list(x_train.index))
        inference_train['fold'] += [fold] * len(list(x_train.index))
        inference_train['n_feats'] += [n_features] * len(list(x_train.index))
        
        inference_valid['ID'] += list(x_valid.index)
        inference_valid['y_true'] += list(response_valid.values.reshape((-1,)))
        inference_valid['y_hat'] += list(y_valid_hat)
        inference_valid['therapy'] += x_valid[therapy_columns].astype(int).apply(therapy_from_dummy, axis=1).tolist()
        inference_valid['threshold'] += [optimal_threshold] * len(list(x_valid.index))
        inference_valid['fold'] += [fold] * len(list(x_valid.index))
        inference_valid['n_feats'] += [n_features] * len(list(x_valid.index))
        
        ########################################################################################################
        # Save fold metadata        
        ########################################################################################################
        result['fold'].append(fold)
        result['n_feats'].append(len(selected_feats))
        result['n_genes'].append(n_features)
        result['auc_train'].append(auc_train)
        result['auc_valid'].append(auc_valid)
        result['optimal_threshold'].append(optimal_threshold)
        result['tn'].append(tn)
        result['fp'].append(fp)
        result['fn'].append(fn)
        result['tp'].append(tp)
        result['ks'].append(ks)
        result['accuracy'].append(metrics['accuracy'])
        result['sensitivity'].append(metrics['sensitivity'])
        result['precision'].append(metrics['precision'])
        result['specificity'].append(metrics['specificity'])
        result['overall_time'].append(overall_time)
        result['lightgbm_train_time'].append(lightgbm_train_time)
        result['baysian_optimization_time'].append(baysian_optimization_time)
        result['dda_time'].append(dda_time)
        result['clinical_normalization_time'].append(clinical_normalization_time)
        result['gene_clustering_time'].append(gene_clustering_time)
        result['genetic_profiling_time'].append(genetic_profiling_time)
        result['feature_selection_time'].append(feature_selection_time)
        result['gene_normalization_time'].append(gene_normalization_time)
        result['optimization_n_folds'].append(optimization_n_folds)
        result['optimization_n_calls'].append(optimization_n_calls)
        result['learning_rate'].append(opt[0])
        result['num_leaves'].append(opt[1])
        result['max_depth'].append(opt[2])
        result['scale_pos_weight'].append(opt[3])
        result['min_child_weight'].append(opt[4])
        result['colsample_bytree'].append(opt[5])
        result['min_split_gain'].append(opt[6])
        result['min_child_samples'].append(opt[7])
        result['subsample'].append(opt[8])
        result['bin_construct_sample_cnt'].append(opt[9])
        result['train_test_distance_avg'].append(train_test_distance_avg)
        result['train_test_distance_std'].append(train_test_distance_std)
        result['train_test_distance_min'].append(train_test_distance_min)
        result['train_test_distance_max'].append(train_test_distance_max)
        result['y_train_hat_min'].append(min(y_train_hat))
        result['y_train_hat_max'].append(max(y_train_hat))
        
        print(fold, n_features, auc_train, auc_valid)

pd.DataFrame(inference_train).to_csv('output/inference_train_10_fold.csv')
pd.DataFrame(inference_valid).to_csv('output/inference_valid_10_fold.csv')
pd.DataFrame(result).to_csv('output/result_10_fold.csv')

Fold 1
1 20 0.9687838325514255 0.716161616161616
1 21 0.7470561503401633 0.6676767676767676
1 22 1.0 0.7464646464646465
1 23 1.0 0.6535353535353536
1 24 1.0 0.6606060606060606
1 25 0.8312950265314032 0.6696969696969697
1 26 0.9993584345803761 0.7494949494949494
1 27 1.0 0.6717171717171717
1 28 1.0 0.6292929292929292
1 29 1.0 0.7151515151515152
1 30 1.0 0.6797979797979798
1 31 0.9999465362150314 0.6212121212121212
1 32 0.9999599021612735 0.6565656565656566
1 33 1.0 0.5696969696969697
1 34 1.0 0.6676767676767676
1 35 1.0 0.592929292929293
1 36 0.9975673977839261 0.8090909090909091
1 37 1.0 0.6262626262626263
1 38 0.9970995896654504 0.7555555555555555
1 39 0.9999866340537579 0.6616161616161615
1 40 1.0 0.6646464646464647
Fold 2
2 20 0.8216211191671314 0.6780748663101605
2 21 0.7715520675643376 0.6978609625668449
2 22 0.9995352295965794 0.6074866310160428
2 23 0.9999734416912331 0.6406417112299465
2 24 0.8152072875999257 0.720855614973262
2 25 1.0 0.6973262032085561
2 26 0.9996945794491807